#####################

We are going to create a "trendy" product recommender to have a guide on what ASOS should produce or have in their online shop, which will tell us: 

a) If our product is "trendy" on Instagram, great! Continue with that selection.

b) If our product is not "trendy" on Instagram, it will point us to other similar products that are more trendy.

#####################

In [4]:
def ASOS_trendy():

    ######## Import libraries ########
    import pandas as pd
    import numpy as np
    from random import randint
    import pickle
   
    ######## Import data ######## 
    
    asos = pd.read_csv('data/asos_clusters.csv')
    
    ######## User input ######## 
    producttype = input('Product Type: ')
    brand  = input('Brand: ')
    colour = input('Colour: ')
    price  = input('Price: £')
     
    
    
    ######## Create a DF ######## 
    input_df = pd.DataFrame({'price': [price],'#search': [producttype], '#brand': [brand] ,'#colour': [colour]}).reset_index(drop=True)
    
    dict_cat = dict(zip(asos['#search'],asos['categories']))
    input_df['categories'] = dict_cat[producttype]
    input_df = input_df[['price', '#search', '#brand', 'categories' ,'#colour']]
    
    input_df['price'] = input_df['price'].astype(float)
    
    ######### Loading the model and scalers from the file using pickle ########

    model = pickle.load(open('models/regression_model.pkl','rb'))
    model_ohe = pickle.load(open('encoders/regression_ohe.pkl','rb'))
    model_minmax = pickle.load(open('transformers/regression_minmax.pkl','rb'))
    
    ###### USING OUR MODEL ######

    X_num_r = input_df[['price']]
    X_cat_r = input_df.drop(columns=['price'],axis =1)
    
    X_num_r_tf = model_minmax.transform(X_num_r)
    X_num_r_tf = pd.DataFrame(X_num_r_tf,columns=X_num_r.columns)
    
    X_cat_r_ohe = model_ohe.transform(X_cat_r).toarray()
    cols_r = model_ohe.get_feature_names_out(input_features=X_cat_r.columns)
    X_cat_r_t = pd.DataFrame(X_cat_r_ohe, columns=cols_r)
    
    #Concat 

    X_r_treated = pd.concat([X_num_r_tf, X_cat_r_t], axis = 1)
    
    ##### PREDICT ####

    

    if model.predict(X_r_treated)[0] > np.quantile(asos['potential_hashtag'], q=0.65):
        return"That's really trendy!"
    else:
        model = pickle.load(open('models/clustering_model.pkl','rb'))
    
    
        input_df['cluster'] = model.predict(X_r_treated)
    
    
        pd.set_option('display.max_colwidth', None)
        return"What you input is not very trendy, we found a similar, more trendy product. Here it is: https://www.google.com/search?q="+asos[(asos['cluster'] == model.predict(X_r_treated)[0])&(asos['trendy'] == 'High')].reset_index().loc[[randint(0,len(asos[(asos['cluster'] == model.predict(X_r_treated)[0])&(asos['trendy'] == 'High')]))]]['name'].str.replace(' ','%20').to_string(index = False)
        

In [6]:
ASOS_trendy()

Price: £30
Product Type: dress
Brand: mango
Colour: leopardprint


'What you input is not very trendy, we found a similar, more trendy product. Here it is: https://www.google.com/search?q=Monki%20poplin%20maxi%20shirt%20dress%20in%20orange'

In [ ]:



Price: £30
Product Type: dress
Brand: mango
Colour: leopardprint